<a href="https://colab.research.google.com/github/Zarak00/Projects/blob/main/Satellite_Reconciliation/Satellite_Reconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from datetime import datetime

In [ ]:
folder_path = "/content/Data"  # Update with the correct folder path
os.chdir(folder_path)


In [ ]:
merged_data = pd.DataFrame()
for file_name in os.listdir():
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        data = pd.read_csv(file_path, delimiter=',')  # Assuming tab-separated data
        merged_data = merged_data.append(data)


<ipython-input-3-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-3-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-3-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-3-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-3-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [ ]:
merged_data.to_csv('/content/Data/2018_data.csv', index=False)


In [ ]:
# Replace 'your_created_file.csv' with the actual path and filename of your CSV file
df1 = pd.read_csv('/content/Data/2018_data.csv')

In [ ]:
# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to convert
df1['Usage'] = df1['Usage'].astype('Int64')

In [ ]:
# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to convert
df1['Usage Unit'] = pd.to_numeric(df1['Usage Unit'], errors='coerce').fillna(0).astype(int)


In [ ]:
# Assuming 'df' is your DataFrame and 'Aircraft' is the name of the column you want to modify
df1['Aircraft'] = df1['Aircraft'].str.replace('-', '')

In [ ]:
# Assuming 'df_2018' is your DataFrame and 'Act Period' is the name of the column
df1['Act Period'] = pd.to_datetime(df1['Act Period']).dt.strftime('%m-%d-%Y')
df1 = df1.sort_values('Act Period')

In [ ]:
df1.sample(5)

,Act Period,Charge Type,Aircraft,Medium Type,Protocol Used,Protocol Billed,Tfc Type,Tfc Direction,Medium / Zone,Station / Site,Avg Unit Rate,Usage,Usage Unit,Tfc Charge
163385,02-01-2019,Traffic,FHBTV,VHF,AOA,POA,ATS,UP,ROW,SNN,0.1,4,0,0.4
118868,08-01-2019,Traffic,HZA24,VHF,POA,POA,AOC,Down,ROW,AHB,0.1,75,0,7.5
98054,04-01-2019,Traffic,XAJVC,VHF,AOA,POA,AOC,Down,ROW,LAS,0.1,12,0,1.2
9999,11-01-2019,Traffic,9HVJV,VHF,POA,POA,AOC,UP,ROW,AUH,0.1,14,0,1.4
142389,08-01-2019,Traffic,PRMLJ,VHF,POA,POA,AOC,Down,ROW,PBI,0.1,3,0,0.3


In [ ]:
# Replace 'your_created_file.csv' with the actual path and filename of your CSV file
df2 = pd.read_csv('/content/Data/2022 SITA Credit FJ analysis.csv')

In [ ]:
df2 = df2.iloc[:, [0]]

In [ ]:
df2.head(5)

,AIRCRAFT
0,0
1,3526
2,10040
3,16402
4,20042


In [ ]:
filtered_df_1 = df1[df1['Charge Type'] == 'Traffic']

In [ ]:
usage_df = filtered_df_1.groupby('Aircraft')['Usage'].sum().reset_index()
usage_df = usage_df.rename(columns={'Aircraft': 'AIRCRAFT', 'Usage': 'Usage'})
usage_df

,AIRCRAFT,Usage
0,010015,1599
1,010040,1201
2,010041,34
3,0134567,3
4,020042,576
...,...,...
4461,ZSJDL,1
4462,ZSMIG,1267
4463,ZZ664,4
4464,ZZ665,571


In [ ]:
filtered_df_2 = df1[df1['Charge Type'] == 'Aircraft']

In [ ]:
avg_unit_rate_df = filtered_df_2.groupby('Aircraft')['Avg Unit Rate'].count().reset_index()
avg_unit_rate_df = avg_unit_rate_df.rename(columns={'Aircraft': 'AIRCRAFT', 'Avg Unit Rate': 'Avg Unit Rate'})
avg_unit_rate_df

,AIRCRAFT,Avg Unit Rate
0,010015,10
1,010040,8
2,010041,2
3,0134567,1
4,020042,5
...,...,...
4502,ZSJDL,1
4503,ZSMIG,6
4504,ZZ664,2
4505,ZZ665,6


In [ ]:
# Merge the two DataFrames based on the 'AIRCRAFT' column
merged_df = df2.merge(usage_df, left_on='AIRCRAFT', right_on='AIRCRAFT', how='left')

# Calculate the new column based on the conditions
merged_df['2018 Usage'] = merged_df['Usage'].fillna(0) * 0.1

# Replace any remaining NaN values with 0
merged_df['2018 Usage'].fillna(0, inplace=True)


In [ ]:
merged_df

,AIRCRAFT,Usage,2018 Usage
0,0,<NA>,0.0
1,3526,<NA>,0.0
2,10040,<NA>,0.0
3,16402,<NA>,0.0
4,20042,<NA>,0.0
...,...,...,...
405,XAGIE,<NA>,0.0
406,XAHNF,98,9.8
407,YLABL,<NA>,0.0
408,ZPBVD,69,6.9


In [ ]:
# Merge the two DataFrames based on the 'A5' column
merged_df_2 = df2.merge(avg_unit_rate_df, left_on='AIRCRAFT', right_on='AIRCRAFT', how='left')

# Calculate the new column based on the conditions
merged_df_2['2018 Subscription'] = merged_df_2['Avg Unit Rate'].fillna(0) * 60

# Replace any remaining NaN values with 0
merged_df_2['2018 Subscription'].fillna(0, inplace=True)



In [ ]:
merged_df_2

,AIRCRAFT,Avg Unit Rate,2018 Subscription
0,0,NaN,0.0
1,3526,NaN,0.0
2,10040,NaN,0.0
3,16402,NaN,0.0
4,20042,NaN,0.0
...,...,...,...
405,XAGIE,NaN,0.0
406,XAHNF,8.0,480.0
407,YLABL,NaN,0.0
408,ZPBVD,6.0,360.0


In [ ]:
# Perform inner join on "AIRCRAFT" column
df3 = pd.merge(merged_df, merged_df_2, on="AIRCRAFT")

# Drop the "Usage" and "Avg Unit Rate" columns from the dataframe
df3 = df3.drop(["Usage", "Avg Unit Rate"], axis=1)

# Create the new "credit" column
df3['credit'] = df3['2018 Usage'] + df3['2018 Subscription']


df3 = df3.drop(df3.index[-1])
# Output the resulting dataframe
df3

,AIRCRAFT,2018 Usage,2018 Subscription,credit
0,0,0.0,0.0,0.0
1,3526,0.0,0.0,0.0
2,10040,0.0,0.0,0.0
3,16402,0.0,0.0,0.0
4,20042,0.0,0.0,0.0
...,...,...,...,...
404,VTAGL,0.0,0.0,0.0
405,XAGIE,0.0,0.0,0.0
406,XAHNF,9.8,480.0,489.8
407,YLABL,0.0,0.0,0.0


In [ ]:
# Calculate the sum of all columns except "AIRPORT"
last_row = df3.drop("AIRCRAFT", axis=1).sum()

# Create a new row using the calculated sum
final_df = df3.append(last_row, ignore_index=True)

<ipython-input-24-d7b2feef7dfb>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = df3.append(last_row, ignore_index=True)


##Change Column Names
- only if you want to otherwisw you can also do it in the excel files

In [ ]:
final_df = final_df.rename(columns={'2018 Usage': '2019 Usage', '2018 Subscription': '2019 Subscription'})

In [ ]:
final_df

,AIRCRAFT,2019 Usage,2019 Subscription,credit
0,0,0.0,0.0,0.0
1,3526,0.0,0.0,0.0
2,10040,0.0,0.0,0.0
3,16402,0.0,0.0,0.0
4,20042,0.0,0.0,0.0
...,...,...,...,...
405,XAGIE,0.0,0.0,0.0
406,XAHNF,9.8,480.0,489.8
407,YLABL,0.0,0.0,0.0
408,ZPBVD,6.9,360.0,366.9


##Download The File
- Change the file name if you feel like

In [ ]:
final_df.to_csv('2019_report.csv', index=False)
